In [1]:
from generation import Llama
import torch
import gc
import os
os.environ['RANK'] = '0'  # Example rank, adjust accordingly
os.environ['WORLD_SIZE'] = '1'  # Example world size, adjust accordingly
os.environ['MASTER_ADDR'] = 'localhost'  # Example master address
os.environ['MASTER_PORT'] = '12355'  # Example master port


In [2]:

llama = Llama.build(ckpt_dir= "../llama3/Meta-Llama-3-8B-Instruct/", tokenizer_path="../llama3/Meta-Llama-3-8B-Instruct/tokenizer.model", max_seq_len= 512, max_batch_size = 4, model_parallel_size=1)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/usr/local/lib/python3.10/dist-packages/torch/__init__.py:690: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


Loaded in 37.20 seconds


In [3]:
cpu = torch.device("cpu")
cuda = torch.device("cuda")

In [4]:
llama.model.to(cuda)
model = llama.model

In [5]:
tokenizer = llama.tokenizer

In [16]:



prompt_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant <|eot_id|>
<|start_header_id|>user<|end_header_id|>

{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

def get_prompt(prompt: str):
    return torch.tensor(tokenizer.encode(prompt_template.format(user_prompt = prompt), eos= False, bos = False), dtype= torch.long).unsqueeze(0)




prompt_tokens = get_prompt("What is France's capital?")


In [17]:
print(tokenizer.decode(prompt_tokens[0].tolist()))


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant <|eot_id|>
<|start_header_id|>user<|end_header_id|>

What is France's capital?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



In [18]:
out_tokens = llama.generate(prompt_tokens, max_gen_len= 1000, temperature = 0.1)

  2%|▏         | 7/437 [00:00<00:16, 25.55it/s]


In [19]:
print(tokenizer.decode(prompt_tokens[0].tolist()+ out_tokens[0][0]))


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant <|eot_id|>
<|start_header_id|>user<|end_header_id|>

What is France's capital?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
The capital of France is Paris.


In [20]:
len(out_tokens[0][0])

7